In [ ]:
# Inputs

# plotting libraries
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
sns.set()
# Define figure sizes
plt.rcParams.update({'figure.figsize': (8, 5), 'figure.dpi': 120})

# Data management libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from dateutil.parser import parse 

# Machine Learning libraries
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_predict
import scipy as sp

# Others
import math
from mltools import forecast_tools as FT
import scipy.stats as st

# Non linear models
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.neural_network import  MLPRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from mltools import model_tools as MT
from mltools import regression_tools as RT

# Load Data

In [ ]:
# Import data
df = pd.read_csv('UnemploymentSpain.dat', sep = '\t')
dates = df['DATE']
df.drop(columns=['DATE'], inplace=True)
df.head()

In [ ]:
# Plotting a time series
fig, ax = plt.subplots()
for col in df.columns.values.tolist():
    ax.plot(col, data=df, label=col, alpha=0.8)
ax.set(title='Time series data', ylabel='Value')
plt.legend()
plt.show()

In [ ]:
# Load time series values
df_ts = df[['TOTAL']]
df_ts

# SARIMA

# Identification process

In [ ]:
# ACF and PACF of the time series -> identify significant lags and order
plt.figure(figsize=[15,15])
FT.ts_display(df_ts)

# No estacionaria en media
# Se observa cierta estacionalidad en PACF, no podemos identificar estacionalidad a simple vista

#  Stabilize the variance

In [ ]:
# Box-cox transformation
lmbda = FT.boxcox_lambda_plot(df_ts, window_width=12)

# No estacionaria en varianza -> Transformacion Box-cox

In [ ]:
# Compute Box Cox
# Indica si queremos transformacion logaritmica
BOX_COX = True
if BOX_COX:
    # SELECIONAR DE GRAFICO ANTERIOR
    lmbda = 0.1608
    z = st.boxcox(df_ts.values[:,0], lmbda = lmbda) #Convert to positive
    #z,lmbda = st.boxcox(df_ts.values[:,0] - min(df_ts.values) + 1) #Convert to positive and automatic selection of lmbda
    z = pd.DataFrame(z, columns=df_ts.columns.values.tolist())
else:
    z = df_ts

FT.ts_display(z)

In [ ]:
#Check Box Cox of transformed series
FT.boxcox_lambda_plot(z, window_width=12)

# Estacionaria en varianza

# Analyze stationarity (mean?)

In [ ]:
# Alternative test - Augmented Dickey Fuller Test
# SI P-VALOR MAYOR QUE 0.05 NECESITAS DIFERENCIAR
result = adfuller(z.values)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

# No estacionaria en media -> Diferenciacion regular

In [ ]:
# Difference of the time series
# Regular
d = 1
# Estacional
D = 1
S = 12 # Seasonality of 12 days

Bz = z
for diff in range(d):
    Bz = Bz.diff().dropna() # drop first NA value
for seas_diff in range(D):
    Bz = Bz.diff(S).dropna() # drop first NA values
plt.figure(figsize=[15,15])
FT.ts_display(Bz,lags=50)

+ Con dif regular se observa periodicidad de 12 empezando en 0 (y en 9 de forma más sutil)
+ ¿Diferenciar primero estacionalmente?
+ Tras diferenciar estacionalmente se observa: Elementos no modelados antes de 100 (crisis 2008) y sobre 250 (covid-19), se pueden modelas con variables impulso (sarimax)

#  Fit SARIMA model

+ Regular: AR(1)
+ Estacional: MA(1)

In [ ]:
# Fit model with estimated order
sarima_fit = SARIMAX(z, 
                    # D: NUM DIFERENCIACIONES HECHAS
                    order=(1,1,0), # Regular components (p, d, q)
                    seasonal_order=(0, 1, 1, 12), # Seasonal components (p, d, q, s)
                    trend= 'n', # Type of trend: ['c','t','n','ct'] --> [constant, linear, no trend, constant and linear]
                    enforce_invertibility=False, 
                    enforce_stationarity=False).fit()

print(sarima_fit.summary())

# Model analysis

+ AIC =  536.253, buscamos que sea lo más bajo posible
+ Todos los coeficientes siginificativos
+ Modelo SARIMA más simple posible que se adecue al comportamiento de la serie temporal
+ Ljung-Box test ?

# Residuals analysis

In [ ]:
# Plot residual error
plt.figure(figsize=[15,15])
FT.check_residuals(pd.DataFrame(sarima_fit.resid.loc[100:]))

+ Ljung-Box test indica que los residuos son independientes (p-valor > alpha -> No rechazamos H_0:{Residuos independientes})
+ Como ya se comentó, existe comportamiento no modelado en la serie temporal
+ Residuos aproximadamente normal, aparecen outliers en la cola derecha, comportamiento no modelado o outliers
+ De ACF y PACF se observa que el residuo no es ruido blanco, existen correlaciones significativas -> No se ha modelado todo el comportamiento
+ Solo a partir de 100?

# Predictions

+ Desde 0, comportamiento extraño al inicio

In [ ]:
#Obtain forecasts for in-sample and out-of-sample
# Inicio y horizonte
start = 200
horizon = 20
end = df_ts.shape[0] + horizon

pred = sarima_fit.get_prediction(start=start, end= end, dynamic=False)
yhat = pred.predicted_mean
yhat_conf_int = pred.conf_int(alpha=0.05)

#Undo Box-cox transform if necessary
if BOX_COX:
    yhat = sp.special.inv_boxcox(yhat, lmbda)
    yhat_conf_int = sp.special.inv_boxcox(yhat_conf_int, lmbda)


plt.figure()
plt.fill_between(yhat_conf_int.index,
                yhat_conf_int.iloc[:, 0],
                yhat_conf_int.iloc[:, 1], color='k', alpha=.2)
plt.plot(df_ts.loc[start:])
plt.plot(yhat)
plt.show()

In [ ]:
# Plot prediction of out_of_sample and confidence intervals
# If using dynamic = True, the forecast are used as real data
horizon = 20
end = df_ts.shape[0] + horizon

# COMANDO PREDICCION
pred = sarima_fit.get_forecast(steps=horizon, dynamic=False)
yhat = pred.predicted_mean
yhat_conf_int = pred.conf_int(alpha=0.05)

#Undo Box-cox transform if necessary
if BOX_COX:
    yhat = sp.special.inv_boxcox(yhat, lmbda)
    yhat_conf_int = sp.special.inv_boxcox(yhat_conf_int, lmbda)


plt.figure(figsize=[15,15])
plt.fill_between(yhat_conf_int.index,
                yhat_conf_int.iloc[:, 0],
                yhat_conf_int.iloc[:, 1], color='k', alpha=.2)
plt.plot(df_ts.loc[1000:])
plt.plot(yhat)
plt.show()

# November 2022 prediction

In [ ]:
# Prediccion para steps = 1 (Escala 1 e6)
predPreBC = sarima_fit.get_forecast(steps=1, dynamic=False).predicted_mean
if BOX_COX:
    pred = sp.special.inv_boxcox(predPreBC, lmbda)

print('Prediction:', round(pred.values[0]), 'parados en Noviembre de 2022')

# SARIMAX

+ Variables impulso: Crisis 2008, Covid-19
+ Visualización + Diferenciaciones realizadas

# Fit SARIMAX model
+ Regular: AR(1)
+ Estacional: MA(1)

In [ ]:
# Crear variables impulso
# Observando df_ts: Covid-19: 229-241
psI = 229
exp = np.zeros(len(df_ts))
exp[psI] = 1

In [ ]:
# 2 opcion: Crecimiento en diferentes etapas, diferentes variables impulso
exp1 = np.zeros(len(df_ts))
exp2 = np.zeros(len(df_ts))
exp1[229] = 1
exp2[230] = 1
exp = np.matrix([exp1, exp2]).T

In [ ]:
# Fit model with estimated order
sarimax_fit = SARIMAX(endog = z, 
                    exog = exp,
                    # D: NUM DIFERENCIACIONES HECHAS
                    order=(1,1,0), # Regular components (p, d, q)
                    seasonal_order=(0, 1, 1, 12), # Seasonal components (p, d, q, s)
                    trend= 'n', # Type of trend: ['c','t','n','ct'] --> [constant, linear, no trend, constant and linear]
                    enforce_invertibility=False, 
                    enforce_stationarity=False).fit()

print(sarimax_fit.summary())

# Model analysis

+ AIC = 465.908 buscamos que sea lo más bajo posible
+ Todos los coeficientes siginificativos
+ Modelo SARIMAX más simple posible que se adecue al comportamiento de la serie temporal
+ Variables impulso: Comportamiento del efecto del covid-19
+ Coeficientes negativos?
+ Ljung-Box test ?

# Residuals analysis

In [ ]:
# Plot residual error
plt.figure(figsize=[15,15])
FT.check_residuals(pd.DataFrame(sarimax_fit.resid.loc[100:]))

+ Ljung-Box test indica que los residuos son independientes (p-valor > alpha -> No rechazamos H_0:{Residuos independientes})
+ Como ya se comentó, existe comportamiento no modelado en la serie temporal
+ Residuos aproximadamente normal, aparecen outliers en la cola derecha, comportamiento no modelado (covid-19) o outliers
+ De ACF y PACF se observa que el residuo no es ruido blanco, existen correlaciones significativas -> No se ha modelado todo el comportamiento
+ Solo a partir de 100?

# Predictions

In [ ]:
#Obtain forecasts for in-sample and out-of-sample
# Inicio y horizonte
start = 200
horizon = 20
end = df_ts.shape[0] + horizon

# Exog variables
exp2Pred = np.concatenate((exp, np.zeros((horizon,2))))

pred = sarimax_fit.get_prediction(start=start, end= end, exog= exp[start:start+horizon+1], dynamic=False)
yhat = pred.predicted_mean
yhat_conf_int = pred.conf_int(alpha=0.05)

#Undo Box-cox transform if necessary
if BOX_COX:
    yhat = sp.special.inv_boxcox(yhat, lmbda)
    yhat_conf_int = sp.special.inv_boxcox(yhat_conf_int, lmbda)


plt.figure()
plt.fill_between(yhat_conf_int.index,
                yhat_conf_int.iloc[:, 0],
                yhat_conf_int.iloc[:, 1], color='k', alpha=.2)
plt.plot(df_ts.loc[start:])
plt.plot(yhat)
plt.show()

In [ ]:
# Plot prediction of out_of_sample and confidence intervals
# If using dynamic = True, the forecast are used as real data
horizon = 20
end = df_ts.shape[0] + horizon

# COMANDO PREDICCION
pred = sarimax_fit.get_forecast(steps=horizon, exog= exp[start:start+horizon], dynamic=False)
yhat = pred.predicted_mean
yhat_conf_int = pred.conf_int(alpha=0.05)

#Undo Box-cox transform if necessary
if BOX_COX:
    yhat = sp.special.inv_boxcox(yhat, lmbda)
    yhat_conf_int = sp.special.inv_boxcox(yhat_conf_int, lmbda)


plt.figure(figsize=[15,15])
plt.fill_between(yhat_conf_int.index,
                yhat_conf_int.iloc[:, 0],
                yhat_conf_int.iloc[:, 1], color='k', alpha=.2)
plt.plot(df_ts.loc[1000:])
plt.plot(yhat)
plt.show()

# November 2022 prediction

In [ ]:
# Prediccion para steps = 1 (Escala 1 e6)
predPreBC = sarimax_fit.get_forecast(steps=1, exog = [0, 0], dynamic=False).predicted_mean
if BOX_COX:
    pred = sp.special.inv_boxcox(predPreBC, lmbda)

print('Prediction:', round(pred.values[0]), 'parados en Noviembre de 2022')

# Comparacion Modelos

+ AIC:
+ Coeficientes significativos:
+ Residuos: Normalidad, test independencia, ruido blanco, outliers
+ Predicciones: 

# Non Linear Model (Time series regression model)

+ Buscamos estacional?

In [ ]:
# Load aditional data (PIB), shift 1, 2, 12 times Total
dfT = df[['TOTAL']].copy()
dfT['TOTAL_lag1'] = dfT['TOTAL'].shift()
dfT['TOTAL_lag2'] = dfT['TOTAL'].shift(2)
dfT['TOTAL_lag12'] = dfT['TOTAL'].shift(3)

dfT.head()

In [ ]:
# Remove missingnvalues from shifting
dfT.dropna(inplace=True)
dfT.head()

In [ ]:
dfT = dfT.astype({'TOTAL_lag1': 'int', 'TOTAL_lag2': 'int', 'TOTAL_lag12': 'int'})

In [ ]:
# Define input and output matrices
INPUTS = ['TOTAL_lag1','TOTAL_lag2', 'TOTAL_lag12']
OUTPUT = 'TOTAL'

X = dfT[INPUTS]
y = dfT[OUTPUT]

In [ ]:
# Divide the data into training and test sets sequentialy
# Create random 80/20 split
X_train = X.iloc[0:round(0.8*X.shape[0])]
X_test = X.iloc[round(0.8*X.shape[0])+1:X.shape[0]]
y_train = y.iloc[0:round(0.8*X.shape[0])]
y_test = y.iloc[round(0.8*X.shape[0])+1:X.shape[0]]

In [ ]:
# Create dataset to store model predictions
dfTR_eval = X_train.copy()
dfTR_eval['Y'] = y_train.copy() 
dfTS_eval = X_test.copy()
dfTS_eval['Y'] = y_test.copy()

In [ ]:
# MLP Regression Model
# Inputs of the model. 
INPUTS_NUM = INPUTS.copy()
INPUTS_CAT = []
INPUTS_MLP = INPUTS_NUM + INPUTS_CAT

param = {'MLP__alpha': [0.0001,0.001,0.01], # Initial value of regularization
         'MLP__hidden_layer_sizes':[(5,),(13,),(20,),(25,)]} # Number of neurons in each hidden layer, enters as tuples
         
"""
# Uncomment the two following lines for training a single model
param = {'MLP__alpha': [0.000001], # Initial value of regularization
         'MLP__hidden_layer_sizes':[(10, 10, 5)]} # Number of neurons in each hidden layer, enters as tuples
"""

    
# Scale data previous to fit and oneHOT
# Prepare numeric variables by scaling values
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
# Prepare the categorical variables by encoding the categories
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore', drop = 'first'))])
# Create a preprocessor to perform the steps defined above
preprocessor = ColumnTransformer(transformers=[
        ('num', numeric_transformer, INPUTS_NUM),
        ('cat', categorical_transformer, INPUTS_CAT)
        ])
pipe = Pipeline(steps=[('preprocessor',preprocessor), # Preprocess the variables when training the model 
                       ('MLP', MLPRegressor(solver='lbfgs', # Update function
                                             activation='logistic', # Logistic sigmoid activation function
                                             #alpha=0.0001, # L2 regularization term
                                             #learning_rate='adaptive', # Type of learning rate used in training
                                             max_iter=450, # Maximum number of iterations
                                             #batch_size=10, # Size of batch when training
                                             #tol=1e-4, # Tolerance for the optimization
                                             #n_iter_no_change=10, # Maximum number of epochs to not meet tol improvement
                                             # random_state=150, # For replication
                                             verbose = True))]) #Print progress

# We use Grid Search Cross Validation to find the best parameter for the model in the grid defined 
nFolds = 10
MLP_fit = GridSearchCV(estimator=pipe, # Structure of the model to use
                       param_grid=param, # Defined grid to search in
                       n_jobs=-1, # Number of cores to use (parallelize)
                       scoring='neg_mean_squared_error', # RMSE 
                       cv=nFolds) # Number of Folds 
MLP_fit.fit(X_train[INPUTS_MLP], y_train) # Search in grid

In [ ]:
# Plot grid error
MT.plotModelGridError(MLP_fit)

In [ ]:
# Model Analysis

mlp = MLP_fit.best_estimator_['MLP']
wts = mlp.coefs_
bias = mlp.intercepts_
actfunc = ['identity',MLP_fit.best_estimator_['MLP'].get_params()['activation'],mlp.out_activation_]
X = MLP_fit.best_estimator_['preprocessor'].transform(X_train) # Preprocess the variables
coefnames = X_train.columns.values.tolist()
X = pd.DataFrame(X, columns=coefnames)
y = pd.DataFrame(y_train, columns=['Y'])
sens_end_layer = 'last'
sens_end_input = False
sens_origin_layer = 0
sens_origin_input = True

In [ ]:
sensmlp = ns.jacobian_mlp(wts, bias, actfunc, X, y)

In [ ]:
sensmlp.summary()

In [ ]:
sensmlp.info()

In [ ]:
sensmlp.plot()

In [ ]:
# Obtain a report of the model based on predictions
dfTR_eval['MLP_pred'] = MLP_fit.predict(X_train)
dfTS_eval['MLP_pred'] = MLP_fit.predict(X_test)

In [ ]:
#Training and test errors

print('Training MAE:',mean_absolute_error(dfTR_eval['Y'], dfTR_eval['MLP_pred']))
print('Test MAE:',mean_absolute_error(dfTS_eval['Y'], dfTS_eval['MLP_pred']))

print('Training RMSE:',math.sqrt(mean_squared_error(dfTR_eval['Y'], dfTR_eval['MLP_pred'])))
print('Test RMSE:',math.sqrt(mean_squared_error(dfTS_eval['Y'], dfTS_eval['MLP_pred'])))

print('Training R2:',r2_score(dfTR_eval['Y'], dfTR_eval['MLP_pred']))
print('Test R2:',r2_score(dfTS_eval['Y'], dfTS_eval['MLP_pred']))

In [ ]:
# Plot predictions of the model
sns.scatterplot(x='TOTAL_lag1', y='Y', data=dfTR_eval, color='black', alpha=0.5)
sns.scatterplot(x='TOTAL_lag1', y='MLP_pred', data=dfTR_eval, color='red', edgecolor='black').set_title('Predictions for training data')
plt.show()

In [ ]:
#Analysis of residuals
RT.plotModelDiagnosis(dfTS_eval, 'MLP_pred', 'Y')

In [ ]:
# Visualize prediction against real value
sns.scatterplot(data=dfTS_eval, x='Y', y='Y', color='black', alpha=0.5)
sns.scatterplot(data=dfTS_eval, x='MLP_pred', y='Y', color='blue', edgecolor='black')
plt.xlabel('Predicted')
plt.show()

In [ ]:
#Compare forecasts in time
plt.figure()
plt.plot('Y', data=dfTS_eval, label='Real')
plt.plot('MLP_pred', data=dfTS_eval, label='Forecast')
plt.legend()
plt.show()

In [ ]:
# Check correlation of residuals
FT.ts_display(dfTS_eval['Y'] - dfTS_eval['MLP_pred'])

# Final Prediction November 2022